Notes:
- [ ] Which predictions change the most from one language to another.
- [ ] Predictions closest to certain axes (slave, master, soldier, child, etc).
- [ ] Can syntax find sentences like: he is like a machine? nonmachine2machine
- [ ] Differences in distributions between the different queries (e.g. machine vs engine): syn labels, metaphoricity, etc.
- [ ] Clusters: slave, female, child

In [ ]:
import pandas as pd
import spacy
import numpy as np
from gensim.models import Word2Vec
from scipy.spatial.distance import cosine
import string
from nltk.corpus import stopwords
import operator

In [ ]:
from utils import explore_preds

In [ ]:
cosine_similarity = lambda x, target : 1 - cosine(x,target)

# Read a word2vec model
w2v_model = Word2Vec.load("./models/word2vec/w2v_1760_1850/w2v_words.model")

In [ ]:
dWordClusters = dict()

dWordClusters["stopwords"] = stopwords.words('english')
dWordClusters["punctuation"] = [x for x in string.punctuation]
dWordClusters["machines"] = ["instrument", "apparatus", "machine", "engine",
                             "engines", "instruments", "tube", "cylinder",
                             "machinery", "machines", "turbine", "turbines",
                             "boiler", "boilers", "dynamo", "dynamos", "motor",
                             "motors", "apparatuses", "accumulator", "accumulators",
                             "compressor", "compressors", "piston", "valve", "pump",
                             "pistons", "valves", "pumps"]
dWordClusters["transport"] = ["car", "cars", "cart", "carts", "boat",
                              "boats", "ship", "ships", "carriage", "carriages", "train", 
                              "trains", "wagon", "wagons", "omnibus", "omnibuses", "ambulance",
                              "ambulances", "steamer", "steamers", "locomotive", "locomotives",
                              "vehicle", "vehicles"]
dWordClusters["workforce"] = ["worker", "workers", "manufacturer", "manufacturers",
                              "slave", "slaves", "servant", "servants", "labourer", 
                              "labourers", "workman", "workmen", "workwoman", "workwomen",
                              "employee", "employees", "craftsman", "craftsmen", "craftswoman",
                              "craftswomen", "tradesman", "tradesmen", "tradeswoman",
                              "tradeswomen", "shopkeeker", "shopkeekers", "merchant", 
                              "merchants", "dealer", "dealers", "trader", "traders",
                              "tradesperson", "tradespersons", "artisan", "technician",
                              "artisans", "technicians", "maker", "makers", "driver",
                              "drivers", "child", "boy", "girl", "children", "boys",
                              "girls", "lad", "man", "men", "woman", "women",
                              "experts", "person", "persons", "people"]
dWordClusters["energy"] = ["energy", "power", "force", "motion", "electricity", "spark",
                           "light", "flame", "powers", "heat", "gas"]
dWordClusters["upperclass"] = ["gentleman", "gentlemen", "king", "kings",
                               "prince", "princes", "lady", "ladies",
                               "princess", "princesses", "queen", "queens"]
dWordClusters["child"] = ["child", "boy", "girl", "children", "youth", "boys",
                          "girls", "lad", "infant", "baby"]
dWordClusters["workanimal"] = ["animal", "animals", "dog", "dogs", "ass", "asses",
                               "mule", "mules", "donkey", "donkeys", "beast", "beasts",
                               "cow", "cows", "ox", "oxen", "pony", "ponies", "creature",
                               "creatures", "bird", "birds", "cat", "cats", "horse",
                               "horses"]
dWordClusters["spiritual"] = ["soul", "souls", "spirit", "spirits", "life", "lives",
                              "idea", "ideas", "art", "arts", "god", "gods", "thought",
                              "thoughts", "mind", "minds", "heart", "hearts", "angel",
                              "angels", "demon", "demons", "virtue", "virtues",
                              "brain", "brains", "instinct", "instincts"]
dWordClusters["body"] = ["body", "bodies", "organism", "tissue", "skeleton",
                         "structure", "frame", "skeletons", "organisms"]
dWordClusters["nature"] = ["air", "water", "flame", "earth", "sun", "planet",
                           "planets", "world", "wind", "storm", "weather"]

In [ ]:
dAvgVectors = dict()
for c in dWordClusters:
    dAvgVectors[c] = explore_preds.w2v_avg_embedding(dWordClusters[c], w2v_model)

In [ ]:
hmd_df = pd.read_pickle("data/hmd_processed/HMD_machine_synparsed_1760_1850_pred_bert.pkl")

In [ ]:
hmd_df.head()

In [ ]:
sim_df = pd.DataFrame(columns=["df_id"] + list(dAvgVectors.keys()))
rows_list = []

for i, row in hmd_df.iterrows():
    preds_test = row.pred_bert_1760_1850
    dClScores = dict()
    for cl in dAvgVectors:
        aggr_values = []
        for pred in preds_test:
            if pred[0] in dWordClusters[cl]:
                aggr_values.append(pred[1])
            elif "#" in pred[0] and cl == "punctuation": # If it is a BERT subword, consider it as punctuation:
                aggr_values.append(pred[1])
            else:
                try:
                    tmp_emb = w2v_model.wv.get_vector(pred[0])
                    aggr_values.append(cosine_similarity(tmp_emb, dAvgVectors[cl]) * pred[1])
                except KeyError:
                    pass

        dClScores[cl] = round(sum(aggr_values), 3)
    dClScores["df_id"] = i
    rows_list.append(dClScores)

sim_df = pd.DataFrame(rows_list)

In [ ]:
hmd_df.merge(sim_df, left_index=True, right_on="df_id").to_csv("hmd_clusters.tsv", sep="\t")